In [33]:
# this is a library that we will use in order for creating break times in order to mimic the human behaviour 
import time

#this library will be used for getting your credential's and don't be shown in the scrip 
from getpass import getpass

#this are the libraries we will be using for interacting with a webpage
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
import random

from bs4 import BeautifulSoup
import re 

# library to open interact with operating system
import os

# You know pandas it's your best buddy 
import pandas as pd

In [34]:
# installing driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        # opening wesite
driver.get('https://www.pnl2027.gov.pt/np4/livrospnl?cat_livrospnl=catalogo_blx#')

In [35]:
#lists of potential parameters

age_list=["0 - 2 anos","3 - 5 anos","6 - 8 anos","9 - 11 anos","12 - 14 anos","15 - 18 anos","maiores de 18 anos"]

theme_list=["Arte","Banda Desenhada","Biografia","Ciência e Tecnologia","Cultura e Sociedade","Ensaio","Literatura","Poesia","Vida Prática"]

period_list=["Antes 2017"]
#period_list=["Antes 2017","2017","2018 - 1.º Sem.","2018 - 2.º Sem.","2019 - 1.º Sem.","2019 - 2.o Sem.","2020 - 1.º Sem.","2020 - 2.o Sem.","2021 - 1.o Sem.","2021 - 2.o Sem.","2022 - 1.o Sem."]

In [36]:
#getting html from settings we want to look for in the page

def parameters_to_select(driver,age,theme,period):

    select_age =Select(driver.find_element(By.ID,'limitbox_1'))
    select_age.select_by_visible_text(age)
    time.sleep(random.randint(1,2))

    #select theme
    select_theme =Select(driver.find_element(By.ID,'limitbox_3'))
    select_theme.select_by_visible_text(theme)
    time.sleep(random.randint(1,2))

    #select period
    select_period =Select(driver.find_element(By.ID,'limitbox_6'))
    select_period.select_by_visible_text(period)
    time.sleep(random.randint(1,2))


    #select search bar 
    #search_bar =Select(driver.find_element(By.CSS_SELECTOR, "input[class^='atividade_filter']"))
    search_bar =driver.find_element(By.XPATH, '//*[@id="search_terms"]')
    search_bar.send_keys(Keys.ENTER)
    time.sleep(12)

    # get the page you are in using page_source attribute 
    html = driver.page_source
    
    
    return html

In [37]:
def get_book_details(soup, res_pages):
    book_titles=[]
    book_author=[]
    book_editor=[]
    page_result=[]
    pnl_edition=[]
    location=2
    for page in res_pages:

        ## get the book details

        tables = soup.find_all("div",attrs={"class":"newsDetailBlx"})

        # book_titles.append([(i.find("div",attrs={"class":"book_title"})).text for i in tables])
        # book_author.append([(i.find("div",attrs={"class":"book_author"})).text for i in tables])
        # book_editor.append([(i.find("div",attrs={"class":"book_editor"})).text for i in tables])

        for i in tables:
            try:
                book_titles.append(i.find("div",attrs={"class":"book_title"}).text)
                book_author.append(i.find("div",attrs={"class":"book_author"}).text)
                book_editor.append(i.find("div",attrs={"class":"book_editor"}).text)
                page_result.append(page)
                pnl_edition.append(period)
            except:
                continue

        ## go to next page
        #page_nr=int(page)
        if location>int(res_pages[-1]):
            break
        else:
            select_page =driver.find_element(By.CSS_SELECTOR,f"a[onclick^='event.preventDefault();changePage({location});']")
            select_page.click()
            time.sleep(10)
            #print(page)
            
            html = driver.page_source
            soup = BeautifulSoup(html)
            location +=1
            #time.sleep(5)
        
    #create database with all the info
    PNL=pd.DataFrame(list(zip(book_titles,book_author,book_editor,page_result,pnl_edition)),columns=["title","author","publisher","res_page","PNLedition"])
    PNL["age_recom"]=age
    PNL["theme"]=theme
    #PNL["PNLedition"]=period
    PNL[['1st_author',"other_authors"]] = PNL.author.str.split(';',n=1, expand=True,)
    #PNL["results_page"]=page
    return PNL
   

In [38]:
## save the database in csv file

def export_csv(df):
    
    return df.to_csv(f"datasets\PNL_{age}_{period}.csv",encoding='latin1')

In [39]:
## parameters we want to export data from

#age_list=["0 - 2 anos","3 - 5 anos","6 - 8 anos","9 - 11 anos","12 - 14 anos","15 - 18 anos","maiores de 18 anos"]
age='6 - 8 anos'

#theme_list=["Arte","Banda Desenhada","Biografia","Ciência e Tecnologia","Cultura e Sociedade","Ensaio","Literatura","Poesia","Vida Prática"]
theme='Literatura'

#period_list=["Antes 2017","2017","2018 - 1.º Sem.","2018 - 2.º Sem.","2019 - 1.º Sem.","2019 - 2.o Sem.","2020 - 1.º Sem.","2020 - 2.o Sem.","2021 - 1.o Sem.","2021 - 2.o Sem.","2022 - 1.o Sem."]
#period='2022 - 1.o Sem.'

In [40]:
for period in period_list:
    html=parameters_to_select(driver,age,theme,period)

        # ##use beautiful soup to extract the data we want
    soup = BeautifulSoup(html)

    ## get the number of result pages

    nr_pages=soup.find_all("a",attrs={"href":"./?"})
    #display(nr_pages)
    nr_pages=list(set([i.text for i in nr_pages]))
    nr_pages.sort()
    if "." in nr_pages[0]:
        ref_number=int(nr_pages[0].strip("..."))
    else:
        ref_number=int(nr_pages[-1])
    page_results=list(range(1,ref_number+1))
    #nr_pages[0].find("onclick")
    book_titles=[]
    book_author=[]
    book_editor=[]

    PNL_df=get_book_details(soup,page_results)

    export_csv(PNL_df)


UnicodeEncodeError: 'latin-1' codec can't encode character '\u201a' in position 60: ordinal not in range(256)